In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v5 import CNN_v5

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v5 = CNN_v5((480, 640, 4), split_into_parts=20)
cnn_v5.def_model()

In [5]:
images = cnn_v5._prepare_images(image_train)
anwsers = cnn_v5._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v5.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v5/weights/cnn_v5_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str, epoch: int = 0):
        self.__path = path
        self.__current_epoch = epoch
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path, 5)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 336s 270us/step - loss: 23.7739 - accuracy: 0.8681 - val_loss: 7.9590 - val_accuracy: 0.8246
Epoch 2/5
1244460/1244460 [==============================] - 328s 264us/step - loss: 6.6932 - accuracy: 0.9337 - val_loss: 4.2192 - val_accuracy: 0.8666
Epoch 3/5
1244460/1244460 [==============================] - 328s 263us/step - loss: 3.0700 - accuracy: 0.9417 - val_loss: 2.0931 - val_accuracy: 0.8803
Epoch 4/5
1244460/1244460 [==============================] - 328s 263us/step - loss: 1.5082 - accuracy: 0.9418 - val_loss: 1.1252 - val_accuracy: 0.8830
Epoch 5/5
1244460/1244460 [==============================] - 327s 263us/step - loss: 0.7879 - accuracy: 0.9399 - val_loss: 0.6725 - val_accuracy: 0.8827


In [14]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/10
1244460/1244460 [==============================] - 339s 272us/step - loss: 62.4959 - accuracy: 0.7317 - val_loss: 34.5997 - val_accuracy: 0.8678
Epoch 2/10
1244460/1244460 [==============================] - 332s 267us/step - loss: 35.9286 - accuracy: 0.8274 - val_loss: 29.0350 - val_accuracy: 0.8785
Epoch 3/10
1244460/1244460 [==============================] - 334s 268us/step - loss: 26.5039 - accuracy: 0.8623 - val_loss: 22.8120 - val_accuracy: 0.8828
Epoch 4/10
1244460/1244460 [==============================] - 333s 267us/step - loss: 21.3654 - accuracy: 0.8828 - val_loss: 18.8705 - val_accuracy: 0.8827
Epoch 5/10
1244460/1244460 [==============================] - 332s 267us/step - loss: 17.9188 - accuracy: 0.8970 - val_loss: 16.0932 - val_accuracy: 0.8805
Epoch 6/10
1244460/1244460 [==============================] - 332s 267us/step - loss: 15.3512 - accuracy: 0.9076 - val_loss: 13.9236 - val_accuracy: 0.8793
Epoch 7/10
1